### Step 1: Set the data location and type

There are two ways to access Azure Blob storage: account keys and shared access signatures (SAS).

To get started, we need to set the location and type of the file.

In [0]:
storage_account_name = "devassignment"
storage_account_access_key = "8YpOLSWDJegnOVlvZzuFpwShdoAPmZpc5Ws4PTz4w6R7sN4WCD+9JgNTs00YgQTxjNfmWVokZ5AE+ASthmNG3g=="

In [0]:
file_location = "wasbs://capstone@devassignment.blob.core.windows.net/models/model_scores.parquet"
file_type = "parquet"

In [0]:
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

### Step 2: Read the data

Now that we have specified our file metadata, we can create a DataFrame. Notice that we use an *option* to specify that we want to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python.

In [0]:
score = spark.read.format(file_type).option("inferSchema", "true").option("header", "true").load(file_location)

In [0]:
score.show()

+-------------+--------+
|        Model|Accuracy|
+-------------+--------+
| logistic_reg|     1.0|
|decision_tree|     1.0|
|random_forest|     1.0|
|  grid_search|     1.0|
+-------------+--------+



### Step 3: Sorting the scores to find the best model

In [0]:
best_models=score.sort(['Accuracy'])

In [0]:
best_models.show()

+-------------+--------+
|        Model|Accuracy|
+-------------+--------+
| logistic_reg|     1.0|
|decision_tree|     1.0|
|random_forest|     1.0|
|  grid_search|     1.0|
+-------------+--------+



In [0]:
dbutils.fs.mkdirs("wasbs://capstone@devassignment.blob.core.windows.net/" + "best_models/")

output_folder = "wasbs://capstone@devassignment.blob.core.windows.net/" + "best_models/best_model.csv"

best_models.repartition(1).write.format("csv") \
    .mode("overwrite") \
    .option("header", "true") \
    .save(output_folder)